# Problème 10 : Calcul de $e$, $\pi$ et $\sqrt{a}$

## A. Nombres à virgule flottante

Avant de faire cette partie, il est nécessaire de lire le document *Représentation des réels : nombres à virgule flottante*.

In [14]:
from math import e, pi
print(e)
print(pi)

2.718281828459045
3.141592653589793


* Pourquoi les constantes `e` et `pi` du module `math` ne contiennent que 15 décimales ?

<font color=blue>
    
Car e et Pi sont deux nombres irrationnel qui ne peuvent être uniquement approximé    
</font>

* Expliquer pourquoi le nombre $0.25$ est représenté de façon exacte en `float` mais pas le nombre $0.2$.

In [15]:
format(0.25, '.40g')

'0.25'

In [16]:
format(0.2, '.40g')

'0.2000000000000000111022302462515654042363'

<font color=blue>

Comme 0.25 est une puissance de 2 soit $2^-2$ alors il est très facile de l'exprimer pour un ordinateur (c'est comme compter sur les doigts pour un humain)
Or 0.2 n'est pas une puissance de 2, ainsi il est approximé ce qui donne cette imprecision
</font>

* La fonction suivante est conçue pour déterminer si une matrice $2 \times 2$ est inversible. Pourquoi donne-t-elle un résultat incorrect pour la matrice $\begin{pmatrix} 0.3 & 1 \\ 0.9 & 3 \end{pmatrix}$ ?

In [17]:
def mat_inversible(M):
    a, b = M[0]
    c, d = M[1]
    return a*d - b*c != 0

mat_inversible([[0.3, 1], [0.9, 3]])

True

<font color=blue>
    
Non ! Cette fonction calcul le mauvais determinant car en python, $0.3 \times 0.3 = 0.89999...$ Ceci est dû à une approximation des floattant en python
Ainsi nous nous retrouvons avec un detreminant égal à 0 en mathématique mais différent de 0 en python
</font>

* On calcule la somme $\displaystyle \sum_{k=1}^n \frac{1}{k^4}$ pour différentes valeurs de $n$. Pourquoi le résultat est-il le même pour $n = 10 000$ et $n= 1 000 000$ ?

In [18]:
for n in [100, 10000, 1000000]:
    s = 0
    for k in range(1, n+1):
        s += 1/k**4
    print(s)

1.0823229053444727
1.082323233710861
1.082323233710861


<font color=blue>
    
D'après le polycopié, lors du calcul de somme, il faut prendre des nombres plus petits pour calculer ces derniers.
</font>

* Proposer une façon de calculer la somme qui évite ce problème (toujours en utilisant des `float`).

In [19]:
for n in [10, 100, 1000]:
    s = 0
    for k in range(1, n+1):
        s += 1/k**4
    print(s)

1.0820365834937566
1.0823229053444727
1.082323233378306


## B. Calcul des décimales de $e$ et $\pi$

Depuis l'introduction de ces constantes (dans l'antiquité pour $\pi$ et au XVIIe siècle pour $e$), les mathématiciens ont cherché à calculer leurs **décimales**, c'est-à-dire leurs chiffres après la virgule, le plus loin possible. De nombreuses méthodes, plus ou moins performantes, ont été développés. Avant l'apparition de l'ordinateur, les calculs étaient naturellement effectués à la main, ce qui n'a pas permis d'aller au-delà de quelques centaines de décimales. Avec l'ordinateur, les calculs ont pu être menée jusqu'à une précision infiniment plus grande.

Nous allons ici calculer les décimales de $e$ et de $\pi$ par des **développements en série** (de la fonction exponentielle pour $e$ et de la fonction $\arctan$ pour $\pi$).

Ces méthodes fournissent des approximations de $e$ et $\pi$ sous forme de nombres rationnels. Nous allons donc utiliser la classe `Rat` programmée dans le Problème 3 plutôt que le type `float`. La classe `Rat` représente un nombre rationnel par un couple d'entiers (le numérateur et le dénominateur). Puisque le type `int` permet de représenter des entiers de taille illimitée (c'est une spécificité de Python), la classe `Rat` permet de représenter tous les rationnels et donc de mener les calculs sans limitation sur la précision.

* Reprendre la classe `Rat` utilisée dans le Problème 3.

In [43]:
import math

class Rat:

    def __init__(self, num, denom=1) -> None:
        pgcd = math.gcd(num, denom)
        self.n = num//pgcd
        self.d = denom//pgcd
        if self.d < 0:
            self.d = -self.d
            self.n = -self.n
        if self.d == 0:
            raise ValueError("Denom = 0")

    def __str__(self) -> str:
        if self.d == 1:
            return str(self.n)
        elif self.d == -1:
            return str(-self.n)
        else:
            return str(self.n) + "/" + str(self.d)


    def __repr__(self):
        """
        >>> rat = Rat(1, 2)
        >>> repr(rat)
        'Rat(1, 2)'
        >>> rat = Rat(-3, 4)
        >>> repr(rat)
        'Rat(-3, 4)'
        """
        if self.d <= 0:
            self.n = -self.n
        return "Rat" + str((self.n, abs(self.d)))


    def __neg__(self):
        """
        >>> rat = Rat(1, 2)
        >>> neg_rat = -rat
        >>> neg_rat.n
        -1
        >>> neg_rat.d
        2
        """
        return Rat(-self.n, self.d)


    def __add__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> sum_rat = rat1 + rat2
        >>> sum_rat.n
        5
        >>> sum_rat.d
        4
        """
        return Rat(self.n * other.d + other.n * self.d, self.d * other.d)


    def __mul__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> mul_rat = rat1 * rat2
        >>> mul_rat.n
        3
        >>> mul_rat.d
        8
        """
        return Rat(self.n * other.n, self.d * other.d)


    def __truediv__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> div_rat = rat1 / rat2
        >>> div_rat.n
        2
        >>> div_rat.d
        3
        """
        return Rat(self.n * other.d, self.d * other.n)


    def __sub__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> sub_rat = rat1 - rat2
        >>> sub_rat.n
        -1
        >>> sub_rat.d
        4
        """
        return self + (-other)
    
    def __eq__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(2, 4)
        >>> rat3 = Rat(3, 4)
        >>> rat1 == rat2
        True
        >>> rat1 == rat3
        False
        """
        if self.n / self.d == other.n /other.d:
            return True
        return False
    
    def __lt__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> rat3 = Rat(2, 3)
        >>> rat1 < rat2
        True
        >>> rat2 < rat3
        False
        """
        if (self.n / self.d) < (other.n / other.d):
            return True
        return False
    
    def __le__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> rat3 = Rat(2, 3)
        >>> rat1 <= rat2
        True
        >>> rat2 <= rat3
        False
        """
        return ((self == other) or (self < other))
    
    def __gt__(self, other):
        """
        >>> rat1 = Rat(1, 2)
        >>> rat2 = Rat(3, 4)
        >>> rat3 = Rat(2, 3)
        >>> rat1 > rat2
        False
        >>> rat2 > rat3
        True
        """
        if (self.n / self.d) > (other.n / other.d):
            return True
            
        return False
    
    def __ge__(self, other):
        """
        """
        return ((self == other) or (self > other))

    def __pow__(self, n):   
        if n == 0 :   
            return Rat(1,1)   
        elif n > 0 :   
            return Rat(self.n**n,self.d**n)   
        else :    
            return Rat(self.d**(-n),self.n**(-n))
    
    def to_dec_string(self, precision):
        """
        >>> Rat(20, 7).to_dec_string(10)
        '2.8571428571'

        >>> Rat(20, 7).to_dec_string(40)
        '2.8571428571428571428571428571428571428571'

        >>> Rat(1, 3).to_dec_string(5)
        '0.33333'

        >>> Rat(-4, 3).to_dec_string(5)
        '-1.33333'

        >>> Rat(7, 2).to_dec_string(4)
        '3.5000'
        """
        abs_numerator = abs(self.n)
        denominator = self.d
        integer_part = abs_numerator // denominator
        decimal_part = abs_numerator % denominator

        result = str(integer_part) + "."

        for _ in range(precision):
            decimal_part *= 10
            digit = decimal_part // denominator
            result += str(int(digit))
            decimal_part %= denominator

        return result


### B.1. Approximation de $e$

On rappelle la formule du **développement de Taylor-Lagrange**.

**Théorème**. Soit $f : \mathbb{R} \to \mathbb{R}$ une fonction de classe $\mathcal{C}^{n+1}$, où $n\in \mathbb{N}$. Soit $a,x \in \mathbb{R}$. Alors il existe un réel $c$ entre $a$ et $x$ tel que :
$$
f(x)=f(a)+f'(a)(x-a)+\frac{f''(a)}{2!}(x-a)^2+\cdots +\frac{f^{(n)}(a)}{n!}(x-a)^n +\frac{f^{(n+1)}(c)}{(n+1)!}
(x-a)^{n+1}.
$$

* Ecrire le développement de Taylor-Lagrange de la fonction exponentielle à l'ordre $n$ pour $a=0$ et $x=1$.

<font color=blue>

$e^1 = e^0 + e^0(1-0) + \frac{e^0}{2!}(1-0)^2 + .. + \frac{e^c}{(n+1)!}(1-0)^{n+1} \\
    = 1 + 1 +\frac{1\times1^2}{2} + ... + \frac{1\times1^n}{n!} + \frac{e^c\times1^{n+1}}{(n+1)!}$
    $e = 2 + \frac{1}{2} + ... + \frac{1}{n!} + \frac{e^c}{(n+1)!}$
    
</font>

* En déduire que, pour tout $n\in \mathbb{N}$,

$$
\left| e -  \left( 1+\frac{1}{1!} +\frac{1}{2!} + \cdots + \frac{1}{n!} \right) \right| \leq \frac{e}{(n+1)!} \leq \frac{3}{(n+1)!}. \qquad(\star)
$$

<font color=blue>
    
$c \in [0, 1]$, donc $e^c \leq e$.

$e - 2 - \frac{1}{2} - ... - \frac{1}{n!} = \frac{e^c}{(n+1)!}$. On a donc $e - 2 - \frac{1}{2} - ... - \frac{1}{n!} \leq \frac{e}{(n+1)!}$

$e - 2 - \frac{1}{2} - ... - \frac{1}{n!} \leq \frac{e}{(n+1)!} \leq \frac{3}{(n+1)!}$

$|e - (1 + \frac{1}{1!} + \frac{1}{2!} + ... + \frac{1}{n!})| \leq \frac{e}{(n+1)!} \leq \frac{3}{(n+1)!}$
    
</font>

L'estimation $(\star)$ montre qu'on peut approcher $e$ par la somme $\displaystyle \sum_{k=0}^n \frac{1}{k!}$. On commet alors une **erreur** inférieure à $\displaystyle \frac{3}{(n+1)!}$, erreur qui devient d'autant plus petite que $n$ est grand.

Grossièrement, si l'erreur commise par une approximation est $< 10^{-(d+1)}$, cette approximation fournit les $d$ premières décimales correctes de la constante approchée. 

* A l'aide d'un script Python, déterminer la plus petite valeur de $n$ qui garantit que $\displaystyle \frac{3}{(n+1)!} < 10^{-101}$.

In [38]:
import math

i = 0
nb = (3)/math.factorial(i + 1)
while nb > 10**-101:
    nb = (3)/math.factorial(i + 1)
    i += 1

print(i)


71


* Ecrire une fonction `approx_e` qui prend en paramètre un entier naturel `n` et renvoie $\displaystyle \sum_{k=0}^n \frac{1}{k!}$ sous forme d'un nombre de type `Rat`. (Indication : quand on a déjà calculé $k !$, il est facile et peu coûteux de calculer $(k+1)!$)

In [39]:
def approx_e(n):
    nb = Rat(0)
    for i in range(n):
        nb += Rat(1, math.factorial(i))
    return nb
approx_e(100)

Rat(31710869445015912176908843526535027555643447320787267779096898248431156738548305814867560678144006224158425966541000436701189187481211772088720561290395499, 11665776930493019085212404857033337561339496033047702683574120486902199999153739451117682997019564785781712240103402969781398151364608000000000000000000000)

* En utilisant `approx_e`, calculer une approximation de $e$ avec une erreur $<10^{-101}$. Afficher les 100 premières décimales de cette approximation.

In [40]:
print(approx_e(71).to_dec_string(100))


2.7182818284590452353602874713526624977572470936999595749669676277240766303535475945713821785251664274


* Vérifier (en cherchant sur internet) que ces 100 premières décimales sont bien les 100 premières décimales de $e$.

<font color=blue>
    
2.7182818284590452353602874713526624977572470936999595749669676277240766303535 = 
2.7182818284590452353602874713526624977572470936999595749669676277240766303535
</font>

### B.2. Approximation de $\arctan (x)$

* Rappeler la définition de la fonction $\arctan$.

<font color=blue>
    
$tan(x) = \frac{sin(x)}{cos(x)}$ et La fonction arctan(x) est l'angle  $\theta$ tel que $\frac{-\Pi}{2} < \theta < \frac{\Pi}{2}$ pour lequel $\tan(\theta) = x$
    
</font>

* Montrer que $\displaystyle \arctan(1)= \frac \pi 4$ (à l'aide de la définition de $\arctan$).

<font color=blue>

Comme $tan(\frac{\Pi}{4}) = 1$ Alors $arctan(1) = \frac{\Pi}{4}$
</font>

On admet l'estimation suivante pour tout $x\in[0,1]$ et pour tout $n \in \mathbb{N}$ :

$$
\left|\arctan(x) - \sum_{k=0}^{n-1} (-1)^k \frac{x^{2k+1}} {2 k + 1} \right| \leq \frac{x^{2n+1}} {2 n + 1}. \qquad (\star \star)
$$

L'estimation $(\star \star)$ montre qu'on peut approcher $\arctan(x)$ par la somme $\displaystyle \sum_{k=0}^{n-1} (-1)^k \frac{x^{2k+1}} {2 k + 1}$. On commet alors une erreur inférieure à $\displaystyle \frac{x^{2n+1}} {2 n + 1}$, erreur qui devient d'autant plus petite que $n$ est grand.

- Ecrire une fonction `approx_arctan` qui prend en paramètres un nombre `x` de type `Rat` et un entier `n`, et qui renvoie $\displaystyle \sum_{k=0}^{n-1} (-1)^k \frac{x^{2k+1}} {2 k + 1}$ sous forme d'un nombre de type `Rat`.

In [64]:
def approx_arctan(x, n):
    d = Rat(0)
    for k in range(n):
        d += Rat((-1)**k) * x**(2*k+1) /Rat((2*k+1))
        
    return d

print((Rat(4) * approx_arctan(Rat(1), 10000)).to_dec_string(15))

3.141492653590043


### B.3. Approximation de $\pi$

#### Formule de Leibniz

Puisque $\displaystyle \arctan(1)= \frac \pi 4$, la fonction `approx_arctan` permet de calculer une approximation de $\pi$ (c'est la formule de Leibniz).

* Quelle est (au pire), l'erreur commise quand on approche $\pi$ par `4 * arctan(Rat(1), n)` ?

<font color=blue>
    
La pire erreur est $4 \times \frac{x^{2n+1}} {2 n + 1}$
</font>

* Si l'on veut garantir une erreur $<10^{-101}$, quelle valeur de `n` faut-il prendre ? (Indication : pas besoin de script, un calcul théorique suffit.)

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

La formule de Leibniz n'est en fait pas très efficace. Il faut calculer beaucoup de termes de la série pour gagner chaque nouvelle décimale (la série converge lentement). C'est pourquoi d'autres formules basées sur le développement en série de $\arctan$ ont été conçues.

#### Formule de Machin

On rappelle la formule suivante (valide pour tous les réels $x$ et $y$  tels que $\tan(x), \tan(y), \tan(x+y)$ soient bien définis) :

$$
\tan(x+y) = \frac{\tan(x)+\tan(y)}{1-\tan(x)\tan(y)}.
$$

* Calculer à la main (sous forme d'un nombre rationnel)
$$ A = \tan\left(2 \arctan\left(\frac{1}{5}\right)\right)$$
puis
$$B = \tan\left(4 \arctan\left(\frac{1}{5}\right)\right).$$ 

* Calculer à la main (sous forme d'un nombre rationnel)

$$C = \tan\left(4 \arctan\left(\frac 1 5 \right)− \frac \pi 4\right)$$ 

* En deduire la formule de Machin :

$$
\frac{\pi}{4} = 4\arctan\left(\frac 1 5\right) - \arctan\left(\frac 1 {239}\right).
$$

<font color=blue>
    
Pour $X$, nous obtenons :

$
X = \tan(2\arctan(\frac{1}{5})) \\
= \tan(\arctan(\frac{1}{5}) + \arctan(\frac{1}{5})) \\
= \frac{2\tan(\arctan(\frac{1}{5}))}{1-\tan(\arctan(\frac{1}{5}))}\\
= \frac{\frac{2}{5}}{1 - (\frac{1}{5})^2}\\
= \frac{5}{12}
$

Pour $Y$, nous trouvons :

$
Y = \tan(4\arctan(\frac{1}{5})) \\
= \tan(2\arctan(\frac{1}{5}) + 2\arctan(\frac{1}{5})) \\
= \frac{2\tan(2\arctan(\frac{1}{5}))}{1-\tan(2\arctan(\frac{1}{5}))^2} \\ 
= \frac{\frac{10}{12}}{1 - \frac{25}{12}} \\
= \frac{1}{5}
$

Pour $Z$, nous calculons :

$
Z = \tan(4\arctan(\frac 1 5) - \frac \pi 4) \\
= \frac{\tan(4\arctan(\frac 1 5)) - \tan(\frac \pi 4)}{(1 + \tan(4\arctan(\frac 1 5)) \times \tan(\frac \pi 4)} \\
= \frac{Y - 1}{1 + Y} \\
= \frac{119}{(119*239)} \\
= \frac{1}{239}
$

Par conséquent, $\frac{\pi}{4} = 4\arctan\left(\frac 1 5\right) - \arctan\left(\frac 1 {239}\right)$ puisque $\arctan$ est l'inverse de $\tan$.

    
</font>

 * A l'aide d'un script Python, déterminer la plus petite valeur de `n` qui garantit que l'erreur commise par `approx_arctan(Rat(1, 5), n)` soit $<10^{-103}$.

 * A l'aide d'un script Python, déterminer la plus petite valeur de `n` qui garantit que l'erreur commise par `approx_arctan(Rat(1, 239), n)` soit $<10^{-103}$.

* En utilisant `approx_arctan(Rat(1, 5), n)` et `approx_arctan(Rat(1, 239), n)` avec les valeurs de `n` précédemment déterminées, calculer une approximation de $\pi$. Afficher les 100 premières décimales de cette approximation.

In [50]:
(Rat(16) * approx_arctan(Rat(1,5),72) - Rat(4) * approx_arctan(Rat(1,239),21)).to_dec_string(100)

'3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679'

* Vérifier (en cherchant sur internet) que ces 100 premières décimales sont bien les 100 premières décimales de $\pi$.

<font color=blue>
    
C'est bon    
</font>

* Essayer maintenant d'obtenir les 1000 premières décimales de $\pi$.

In [55]:
print(((Rat(16) * approx_arctan(Rat(1,5),72) - Rat(4) * approx_arctan(Rat(1,239),21)).to_dec_string(1000)))

3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706798162478513934506898440362801792706010179987216806726188740140466033567581311679376075335101609659171030644576233653027450257182803484658351860927270133809030914436823660262931162576284703194589395221866245992710817555393680237554917047871708932985106840785074833639247080859264327721882027979677397953754604196915619381410505600288856897761875941052867609089114345150157869223684881643245943313338421018485091403977277400743970527492816321894223953257584787737337170568053925027217102844351208765657302025589127695185039186644597240030541171074757870137431100579097277905612641495178817964173941740654985445918326928220945355416048444887050935562696866696019631573868714587428709669938320262709342763703083411915986340840811600780101637670651427561210644918709616109476626897981140129879434807832856065243548777350380397289995989938283022000030335652536487563076892056301947868188277818071411096251

* Combien de décimales de $\pi$ sont connues actuellement ?

<font color=blue>
    
62 800 milliards de decimales après la virgule nous connaissons actuellement    
</font>

## C. Calcul des racines carrées

Pour calculer la racine carrée d'un nombre, nous allons utiliser la **méthode de Newton**. Il s'agit d'une **méthode itérative** d'approximation des solutions d'une équation du type $f(x)=0$, où $f$ est une fonction réelle de classe $C^1$. Elle consiste, en partant d'une valeur $x_0$, à calculer les termes de la suite récurrente définie par la relation
\begin{equation*}
\tag{$\star$}
x_{k+1}=x_{k}-\frac{f(x_{k})}{f'(x_{k})}.
\end{equation*}

Si la valeur $x_0$ est bien choisie, la suite $(x_k)_{k\in\mathbb{N}}$ converge vers une solution de l'équation $f(x)=0$. En pratique, on arrête le calcul de la suite dès que l'écart entre $x_k$ et $x_{k+1}$ est devenu suffisamment faible.

Soit $a$ est un réel strictement positif. La quantité $\sqrt{a}$ est solution de l'équation $x^2-a = 0$. Pour cette équation, la relation de récurrence de la méthode de Newton s'écrit :
\begin{equation*}
\tag{$\star \star$}
x_{k+1} = \frac{1}{2}\left(x_{k}+\frac{a}{x_{k}}\right).
\end{equation*}

* Vérifier que la relation ($\star\star$) est bien la relation de récurrence de la méthode de Newton pour l'équation $x^2-a = 0$.

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

### C.1. Implémentation de la méthode

Contrairement à la partie précédente, nous allons ici utiliser des `float`. Nous cherchons pas à obtenir une précision supérieure à celle fournie par les `float`.

* Ecrire une fonction `ceilsqrt` qui calcule le plus petit entier naturel supérieur à la racine carrée d'un nombre donné (autrement dit, le plus petit entier naturel dont le carré est supérieur à un nombre donné). Cette fonction ne devra pas utiliser la fonction `sqrt` du module `math`.

In [58]:
def ceilsqrt(n):
    for i in range(n-1, -1, -1):
        if i**2 <= n:
            return i
ceilsqrt(100)

10

- Ecrire une fonction `mysqrt` qui calcule la racine carrée d'un nombre flottant positif `x` en utilisant la méthode de Newton. On choisira comme valeur initiale `ceilsqrt(x)`. On arrêtera les itérations dès qu'on dépassera 100 itérations ou dès que l'erreur relative entre les valeurs de deux itérations successives sera inférieure à $2 \times 10^{-16}$ (autrement dit, dès que $|x_{k+1}-x_k|<2 \times 10^{-16} |x_{k+1}|$).

In [59]:
def mysqrt(nb):
    x = ceilsqrt(nb)
    n_iter = 0
    while n_iter != 100:
        y = 0.5 * (x + nb/x)
        if abs(y - x) < 2e-16 * abs(y):
            break
        n_iter += 1
        x = y
    return y

- Calculer l'erreur relative entre la valeur calculée par cette fonction `mysqrt` et la fonction `sqrt` du module `math` pour $\sqrt{5},\sqrt{10},\ldots,\sqrt{100}$.

In [62]:
from math import sqrt

nombres = [2, 5, 10, 100]
for n in nombres:
    x1, x2 = mysqrt(n), sqrt(n)
    print("erreur relative :", x2-x1)

erreur relative : 2.220446049250313e-16
erreur relative : 0.0
erreur relative : 4.440892098500626e-16
erreur relative : 0.0


- Ecrire une variante de la fonction `mysqrt` qui affiche la valeur calculée à chaque itération. Tester cette fonction sur le calcul de $\sqrt{2}$. Combien de décimales correctes (par rapport à la valeur exacte de $\sqrt{2}$) possède la valeur approchée fournie par la méthode à chaque itération ? Comment évolue ce nombre de décimales correctes entre deux itérations ?

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

### C.2. Extension au calcul des racines n-ièmes

- En s'inspirant de ce qui a été fait pour le calcul de la racine carrée, écrire une fonction `nthroot` qui calcule la racine n-ième en utilisant la méthode de Newton.

- Calculer l'erreur relative entre la valeur calculée par cette fonction `nthroot` et celle calculée à l'aide de la fonction `pow` du module `math` pour $\sqrt[3]{5},\sqrt[3]{10},\ldots,\sqrt[3]{100}$.

### C.3. Analyse théorique (facultative)

Soit $a>0$. Soit $x_0 \ge \sqrt{a}$. On note $(x_k)_{k\in\mathbb{N}}$ la suite définie par la relation ($\star\star$) en partant de $x_0$.

* Justifier brièvement que les termes de la suite $(x_k)_{k\in\mathbb{N}}$ sont strictement positifs.

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

* Exprimer $x_{k+1} - \sqrt{a}$ en fonction de $\left(x_{k} - \sqrt{a}\right)^2$ pour tout $k\in \mathbb{N}$.

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

* Montrer que $x_k \geq \sqrt{a}$ pour tout $k\in \mathbb{N}$.
* Montrer que la suite $(x_k)_{k\in\mathbb{N}}$ est décroissante.
* En déduire qu'elle converge vers $\sqrt{a}$.

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>

 * Soit $k\in \mathbb{N}^*$. On suppose que $|x_{k}-\sqrt{a}|\leq 10^{-n}$, où $n \in \mathbb{N}^*$. Montrer que 
 $$ |x_{k+1}-\sqrt{a}| \leq \frac{10^{-2n}}{2 \sqrt{a}}.$$

<font color=blue>
    
-- *Écrire la réponse ici.* --
    
</font>